# FLITS Seed Preview UI (scattering)
Adjust model parameters to visually align with the data, then click Save to write an init-guess JSON for the pipeline.

In [1]:
import json
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatLogSlider, FloatSlider, IntSlider, Button, HBox, VBox, ToggleButtons, Output, Text
import ipywidgets as widgets
import sys

repo_root = "/Users/jakobfaber/Documents/research/caltech/ovro/dsa110/chime_dsa_codetections/FLITS"
sys.path.insert(0, str(Path(repo_root)/"scattering"))

# Inputs
data_path = Text(value='synthetic_dynspec.npy', description='Data NPY:')
out_seed = Text(value='flits_init_guess.json', description='Seed JSON:')
mode = ToggleButtons(options=['single','multi'], value='single', description='Mode:')
ncomp = IntSlider(value=1, min=1, max=5, step=1, description='components')

# Load data
def load_data(p):
    arr = np.load(p)
    # Build default axes (ms, GHz) based on array size for visualization
    t = np.linspace(-10.0, 10.0, arr.shape[1])
    f = np.linspace(0.4, 0.8, arr.shape[0])
    return arr, t, f

data, time, freq = load_data(data_path.value)

# Shared sliders
s_gamma = FloatSlider(value=-1.6, min=-5, max=1, step=0.05, description='gamma')
s_tau = FloatLogSlider(value=0.5, base=10, min=-3, max=2, step=0.01, description='tau_1ghz (ms)')
s_alpha = FloatSlider(value=4.4, min=0.5, max=8.0, step=0.05, description='alpha')
s_ddm = FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.01, description='delta_dm')

# Per-component builders
def make_comp_sliders(i):
    return (
        FloatLogSlider(value=1.0, base=10, min=-3, max=3, step=0.01, description=f'c0_{i}'),
        FloatSlider(value=0.0, min=time.min(), max=time.max(), step=(time[1]-time[0]), description=f't0_{i}'),
        FloatLogSlider(value=0.2, base=10, min=-3, max=2, step=0.01, description=f'zeta_{i}'),
    )

comp_widgets = [make_comp_sliders(1)]

def rebuild_components(change=None):
    global comp_widgets
    comp_widgets = [make_comp_sliders(i) for i in range(1, ncomp.value+1)]
    comps_box.children = [VBox([w[0], w[1], w[2]]) for w in comp_widgets]

ncomp.observe(rebuild_components, names='value')
comps_box = HBox([])
rebuild_components()

out = Output()

def simulate_single(gamma, tau1, alpha, ddm, c0, t0, zeta):
    # Simple preview: frequency-independent Gaussian+exp tail time-profile
    # Not exact pipeline model, but good for visual alignment.
    tt = time[None, :]
    profile = c0 * np.exp(-0.5*((tt - t0)/zeta)**2) * np.exp(-np.maximum(0, (tt - t0)) / max(tau1, 1e-6))
    # crude alpha scaling across band
    scale = (freq[:, None] / 1.0) ** (-alpha)
    return profile * scale

def draw(*_):
    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(1,2, figsize=(10,4))
        extent=[time[0], time[-1], freq[0], freq[-1]]
        ax[0].imshow(data, origin='lower', aspect='auto', extent=extent, cmap='plasma'); ax[0].set_title('Data')
        if mode.value=='single':
            c0,t0,zeta = comp_widgets[0]
            sim = simulate_single(s_gamma.value, s_tau.value, s_alpha.value, s_ddm.value, c0.value, t0.value, zeta.value)
        else:
            sim = 0
            for cw in comp_widgets:
                c0,t0,zeta = cw
                sim = sim + simulate_single(s_gamma.value, s_tau.value, s_alpha.value, s_ddm.value, c0.value, t0.value, zeta.value)
        ax[1].imshow(sim, origin='lower', aspect='auto', extent=extent, cmap='plasma'); ax[1].set_title('Model')
        plt.show()

def save_seed(_):
    if mode.value=='single':
        c0,t0,zeta = comp_widgets[0]
        payload = {
            'model_key':'M3',
            'c0': float(c0.value),
            't0': float(t0.value),
            'gamma': float(s_gamma.value),
            'zeta': float(zeta.value),
            'tau_1ghz': float(s_tau.value),
            'alpha': float(s_alpha.value),
            'delta_dm': float(s_ddm.value),
        }
    else:
        comps=[]
        for i,cw in enumerate(comp_widgets, start=1):
            c0,t0,zeta = cw
            comps.append({'c0':float(c0.value), 't0':float(t0.value), 'zeta':float(zeta.value)})
        payload = {
            'model_key':'M3_multi',
            'shared':{
                'gamma': float(s_gamma.value),
                'tau_1ghz': float(s_tau.value),
                'alpha': float(s_alpha.value),
                'delta_dm': float(s_ddm.value),
            },
            'components': comps
        }
    Path(out_seed.value).write_text(json.dumps(payload, indent=2))
    with out:
        print('Wrote seed to', Path(out_seed.value).resolve())

btn_draw = Button(description='Update Preview')
btn_save = Button(description='Save Seed')
btn_draw.on_click(draw)
btn_save.on_click(save_seed)

ui = VBox([HBox([data_path, out_seed]), HBox([mode, ncomp]),
            HBox([s_gamma, s_tau, s_alpha, s_ddm]), comps_box, HBox([btn_draw, btn_save]), out])
display(ui)
draw()


FileNotFoundError: [Errno 2] No such file or directory: 'synthetic_dynspec.npy'